<a href="https://colab.research.google.com/github/neelagarwal4321/RAG-LLM_RESEARCH/blob/main/logistic_practice_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Naive Bayes Classification

In [ ]:
import math

class NaiveBayes:
    def fit(self, X, y):
        self.classes = list(set(y))
        self.params = {}

        for c in self.classes:
            X_c = [x for x,label in zip(X,y) if label==c]
            self.params[c] = {
                "mean": [sum(f)/len(f) for f in zip(*X_c)],
                "var": [sum((xi-mu)**2 for xi in f)/len(f) for f,mu in zip(zip(*X_c), [sum(f)/len(f) for f in zip(*X_c)])],
                "prior": len(X_c)/len(X)
            }

    def gaussian(self, x, mean, var):
        return (1/math.sqrt(2*math.pi*var)) * math.exp(-(x-mean)**2/(2*var))

    def predict_one(self, x):
        posteriors = {}
        for c in self.classes:
            prior = math.log(self.params[c]["prior"])
            cond = sum(math.log(self.gaussian(xi, mu, var)) for xi,mu,var in zip(x,self.params[c]["mean"],self.params[c]["var"]))
            posteriors[c] = prior + cond
        return max(posteriors, key=posteriors.get)

    def predict(self, X):
        return [self.predict_one(x) for x in X]


Bayesian Logistic Regression

In [ ]:
import random

class BayesianLogisticRegression:
    def __init__(self, lr=0.01, epochs=1000, lam=1.0):
        self.lr = lr
        self.epochs = epochs
        self.lam = lam  # regularization strength (Bayesian prior)

    def sigmoid(self, z):
        return 1/(1+math.exp(-z))

    def fit(self, X, y):
        n, m = len(X), len(X[0])
        self.w = [random.random() for _ in range(m)]
        self.b = 0

        for _ in range(self.epochs):
            for i in range(n):
                z = sum(self.w[j]*X[i][j] for j in range(m)) + self.b
                p = self.sigmoid(z)
                error = y[i] - p

                # Update with prior penalty
                for j in range(m):
                    self.w[j] += self.lr * (error*X[i][j] - self.lam*self.w[j])
                self.b += self.lr * error

    def predict_one(self, x):
        z = sum(self.w[j]*x[j] for j in range(len(x))) + self.b
        return 1 if self.sigmoid(z) >= 0.5 else 0

    def predict(self, X):
        return [self.predict_one(x) for x in X]


Support Vector Machine

In [ ]:
class SVM:
    def __init__(self, lr=0.01, epochs=1000, lam=0.01):
        self.lr = lr
        self.epochs = epochs
        self.lam = lam

    def fit(self, X, y):
        n, m = len(X), len(X[0])
        self.w = [0.0 for _ in range(m)]
        self.b = 0

        # Convert labels {0,1} → {-1,1}
        y_ = [1 if label==1 else -1 for label in y]

        for _ in range(self.epochs):
            for i in range(n):
                condition = y_[i]*(sum(self.w[j]*X[i][j] for j in range(m)) + self.b)
                if condition >= 1:
                    for j in range(m):
                        self.w[j] -= self.lr * (2*self.lam*self.w[j])
                else:
                    for j in range(m):
                        self.w[j] += self.lr * (y_[i]*X[i][j] - 2*self.lam*self.w[j])
                    self.b += self.lr * y_[i]

    def predict_one(self, x):
        return 1 if sum(self.w[j]*x[j] for j in range(len(x))) + self.b >= 0 else 0

    def predict(self, X):
        return [self.predict_one(x) for x in X]

In [ ]:
# Split dataset
X = [row[:2] for row in dataset]
y = [row[2] for row in dataset]

# Naive Bayes
nb = NaiveBayes()
nb.fit(X,y)
print("Naive Bayes predictions:", nb.predict(X))

# Bayesian Logistic Regression
blr = BayesianLogisticRegression(lr=0.01, epochs=2000, lam=0.1)
blr.fit(X,y)
print("Bayesian Logistic Regression predictions:", blr.predict(X))

# SVM
svm = SVM(lr=0.01, epochs=2000, lam=0.01)
svm.fit(X,y)
print("SVM predictions:", svm.predict(X))

Naive Bayes predictions: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Bayesian Logistic Regression predictions: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
SVM predictions: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
